In [ ]:
!pip install numpy
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

# Or try the preview build (unstable)
!pip install tf-nightly

# Importing data from github

In [ ]:
# NOTE: YOU SHOULD ONLY NEED TO RUN THIS STEP THE FIRST TIME IN A SESSION
import tensorflow as tf

# get the data from github and unzip
!wget https://raw.githubusercontent.com/andrsn/data/main/speechImageData.zip
!unzip -q /content/speechImageData.zip


# Pre-process data into training and validation sets, using Keras dataset objects

In [ ]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    directory='/Users/abdulrasheed/Desktop/Robotics/Deep Learning Assign/Python/speechImageData/TrainData',
#     directory='/content/speechImageData - Copy/TrainData',
    labels='inferred',
    color_mode="grayscale",
    label_mode='categorical',
    batch_size=128,
    image_size=(98, 50)
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    directory='/Users/abdulrasheed/Desktop/Robotics/Deep Learning Assign/Python/speechImageData/ValData',
#     directory='/content/speechImageData - Copy/ValData',
    labels='inferred',
    color_mode="grayscale",
    label_mode='categorical',
    batch_size=128,
    image_size=(98, 50)
)

# Extract input-output data into arrays, which can be more useful

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Input, Conv2D, MaxPooling2D, Flatten, Softmax
from keras import optimizers, regularizers

#####

# from keras.wrappers.scikit_learn import KerasClassifier


# Extract the  training input images and output class labels
x_train = []
y_train = []
for images, labels in train_ds.take(-1):
    x_train.append(images.numpy())
    y_train.append(labels.numpy())

x_train = np.concatenate(x_train, axis=0)
y_train = np.concatenate(y_train, axis=0)


print(y_train)

# Extract the validation input images and output class labels
x_val = []
y_val = []
for images, labels in val_ds.take(-1):
    x_val.append(images.numpy())
    y_val.append(labels.numpy())

x_val = np.concatenate(x_val, axis=0)
y_val = np.concatenate(y_val, axis=0)

bayes_train_x = x_train
bayes_train_y = y_train
bayes_val_x = x_val
bayes_val_y = y_val


print(y_val)


# Defining the classes

In [ ]:
# print data info
print('Number of training data:' , len(x_train))
print('Number of testing data:' , len(x_val))

# define class labels
class_names = ['background', 'down', 'go', 'left', 'no',
               'off', 'on', 'right', 'stop', 'unknown','up','yes']
num_classes = 12 # number of classes

# display example image
plt.figure()
plt.imshow(x_train[0])
#plt.colorbar()
plt.grid(False)
plt.show()

# Data pre-processing

In [ ]:
# Normalising image data
x_train = x_train/255.0
x_val = x_val/255.0


# display some sample images
plt.figure(figsize=(10,10))
for i in range(25):
    index = np.where(y_train[i] == 1)[0]
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[index[0]])
plt.show()

# Model 1 (Convolutional Neural Networks)


The model is build total of 12 layers in which convolutional layer , Batch normalization layer , Activation layer and Max Pooling layer are of 4 sets. Tuning the Number of filters to , and finally, validation accuracy of ' -- ' with number of filters for the convolutional layer are set to '64',Batch size to '16' optimizer learning rate to '0.004' .

In [ ]:
# Model averaging for model1

import numpy as np
import random
# define number of samples
nsamples = len(x_train)
# create data index
data_index = list(range(1,nsamples))
# create random index using sampling with replacement
idx = random.choices(data_index, k=nsamples)
# initialise data set 1
x1 = np.zeros([nsamples,98,50,1])
y1 = np.zeros([nsamples,12])
# resample training data with replacement
for i in range(nsamples):
  x1[i] = x_train[idx[i],:,:,:]
  y1[i] = y_train[idx[i],:]

# Model 1 - Baseline Model(Convolutional Neural Network)


# number of convolutional filters
num_filters = 64


# define model
model = Sequential()

model.add(Input(shape=(98,50,1)))

for i in range(4):
  model.add(Conv2D(num_filters, kernel_size =(3, 3), padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size =(2, 2), strides=(2, 2), padding= 'same'))



model.add(Conv2D(num_filters, kernel_size =(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size =(12, 1), strides=(1, 1), padding= 'same'))

# model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(num_classes))
model.add(Softmax())

# set the optimization options and compile the model
opt = optimizers.Adam(learning_rate=0.004)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])



# print out the model summary
model.summary()




# Model training

In [ ]:
model1 = model.fit(x1, y1, batch_size=16, epochs=10, validation_split=0.1)

# Installing scikeras 

In [ ]:
!pip install scikeras

# Model 2 - 3x2 Grid Search (Convolutional Neural Networks)

1. For batch_size=128,num_layers=[2,3,4],num_filters=[32, 64]validation accuracy is 0.2784
2. For batch_size=64,num_layers=[2,3,4],num_filters=[32, 64]validation accuracy is 0.5628
3. For batch_size=32,num_layers=[2,3,4],num_filters=[32, 64]validation accuracy is 0.6114
4. For batch_size=32,num_layers=[2,3,4],num_filters=[32, 64]validation accuracy is 0.6285

In [ ]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
import pandas as pd

# Model averaging for Model 2
import numpy as np
import random
# define number of samples
nsamples = len(x_train)
# create data index
data_index = list(range(1,nsamples))
# create random index using sampling with replacement
idx = random.choices(data_index, k=nsamples)
# initialise data set 1
x2 = np.zeros([nsamples,98,50,1])
y2 = np.zeros([nsamples,12])
# resample training data with replacement
for i in range(nsamples):
  x2[i] = x_train[idx[i],:,:,:]
  y2[i] = y_train[idx[i],:]


# Model 2 - Grid Search 

def create_model(num_layers,num_filters):
    grid_model = Sequential()
    grid_model.add(Input(shape=(98,50,1)))
    for i in range(num_layers-1):
        grid_model.add(Conv2D(num_filters, kernel_size=(3, 3), padding='same'))
        grid_model.add(BatchNormalization())
        grid_model.add(Activation('relu'))
        grid_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    grid_model.add(Conv2D(num_filters, kernel_size=(3, 3), padding='same'))
    grid_model.add(BatchNormalization())
    grid_model.add(Activation('relu'))

    grid_model.add(MaxPooling2D(pool_size=(12, 1), strides=(1, 1), padding='same'))

    grid_model.add(Flatten())

    grid_model.add(Dense(num_classes,activation='softmax'))


    # Set the optimization options and compile the model
    opt = optimizers.Adam(learning_rate=0.004)
    grid_model.compile( optimizer=opt,loss="categorical_crossentropy", metrics=["accuracy"])

    return grid_model


# Create KerasClassifier
# build_fn
grid_model = KerasClassifier(model=create_model, epochs=10, batch_size=32, verbose=1,num_layers=[3, 4, 5, 6],num_filters=[32, 64, 128, 256])

# Define parameters for grid search
param_grid = {
    'num_layers':[3, 4, 5, 6],
    'num_filters': [32, 64, 128, 256]
}


# Create GridSearchCV
grid = GridSearchCV(estimator=grid_model, param_grid=param_grid,n_jobs=1, verbose=1)

# Fit the model
model2 = grid.fit(x2, y2,validation_data=(x_val, y_val))
# grid.predict(x_val)
# first, output the best performing parameters
print(grid.best_params_)
# output is in
df = pd.DataFrame(grid.cv_results_)
print(df)


# Model 3 -  Bayesian optimization ( Convolutional Neural Network )

With Bayesian optimization tried tuning the parameters and got the below accuracies:
1. Batch size = 16, hidden layers =[2,3,4], Filters = [32,64] the validation accuracy is 0.7612.
2. Batch size = 32, hidden layers =[2,3,4], Filters = [32,64] the validation accuracy is 0.7463.
3. Batch size = 32, hidden layers =[2,3,4], Filters = [64,128] the validation accuracy is 0.7214
5. Batch size = 128,  hidden layers =[2,3,4], Filters =[32,64] the validation accuracy is 0.4030
6. Batch size = 64,  hidden layers =[2,3,4], Filters =[32,64] the validation accuracy is 0.4876

Finally, Batch size = 32,  hidden layers =[2,3,4], Filters =[32,64] the validation accuracy is 0.7711 has good validation accuracy.

In [ ]:
pip install git+https://github.com/hyperopt/hyperopt.git

In [ ]:
from hyperopt import fmin, tpe, hp

# Model averaging for Model 2
import numpy as np
import random
# define number of samples
nsamples = len(x_train)
# create data index
data_index = list(range(1,nsamples))
# create random index using sampling with replacement
idx = random.choices(data_index, k=nsamples)
# initialise data set 1
x3 = np.zeros([nsamples,98,50,1])
y3 = np.zeros([nsamples,12])
# resample training data with replacement
for i in range(nsamples):
  x3[i] = x_train[idx[i],:,:,:]
  y3[i] = y_train[idx[i],:]


# Model 3 - Bayesian Optimization  


space = {'num_hidden_layers': hp.choice('num_hidden_layers', [3, 4, 5, 6]),
'num_hidden_units': hp.choice('num_hidden_units', [32, 64, 128, 256]),}
# Define the objective function to minimize

def objective(params):
# Build the Keras model with the given hyperparameters
  bayes_model = Sequential()
  bayes_model.add(Input(shape=(98,50,1)))

  for i in range(params['num_hidden_layers']-1):
    bayes_model.add(Conv2D(params['num_hidden_units'], kernel_size =(3, 3), padding='same'))
    bayes_model.add(BatchNormalization())
    bayes_model.add(Activation('relu'))
    bayes_model.add(MaxPooling2D(pool_size =(2, 2), strides=(2, 2), padding= 'same'))
  bayes_model.add(Conv2D(params['num_hidden_units'], kernel_size =(3, 3), padding='same'))
  bayes_model.add(BatchNormalization())
  bayes_model.add(Activation('relu'))
  bayes_model.add(MaxPooling2D(pool_size =(12, 1), strides=(1, 1), padding= 'same'))
  bayes_model.add(Flatten())
  bayes_model.add(Dense(num_classes))
  bayes_model.add(Softmax())
# set the optimization options and compile the model
  opt = optimizers.Adam(learning_rate=0.004)
  bayes_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# Train the model and evaluate on the test set
  # bayes_model.fit(x_train, y_train, epochs=10, batch_size=16, verbose=0)
  bayes_model.fit(x3, y3, batch_size=128, epochs=10, validation_split=0.1)
  loss, accuracy = bayes_model.evaluate(x_val, y_val, verbose=0)
  return {'loss': -accuracy, 'status': 'ok'}

# Run the optimization
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50)


print('best values: ',best)

In [ ]:
# Extract the best hyperparameters
# best_num_hidden_layers = space['num_hidden_layers'][best['num_hidden_layers']]
# best_num_hidden_units = space['num_hidden_units'][best['num_hidden_units']]
best_num_hidden_layers = best['num_hidden_layers']
best_num_hidden_units = best['num_hidden_units']
print('besnaksd',best)
# Build the best Bayesian model using the best hyperparameters
best_bayes_model = Sequential()
best_bayes_model.add(Input(shape=(98, 50, 1)))

for _ in range(best_num_hidden_layers - 1):
    best_bayes_model.add(Conv2D(best_num_hidden_units, kernel_size=(3, 3), padding='same'))
    best_bayes_model.add(BatchNormalization())
    best_bayes_model.add(Activation('relu'))
    best_bayes_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

best_bayes_model.add(Conv2D(best_num_hidden_units, kernel_size=(3, 3), padding='same'))
best_bayes_model.add(BatchNormalization())
best_bayes_model.add(Activation('relu'))
best_bayes_model.add(MaxPooling2D(pool_size=(12, 1), strides=(1, 1), padding='same'))
best_bayes_model.add(Flatten())
best_bayes_model.add(Dense(num_classes))
best_bayes_model.add(Softmax())

# Compile the model
opt = optimizers.Adam(learning_rate=0.004)
best_bayes_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


# Model averaging Voting


In [ ]:
from sklearn.metrics import accuracy_score
from scipy.stats import mode

# Step 1: Train multiple models on random splits of the data
models = [model1,model2,model3]


# Step 2: Predict with each trained model
predictions = []
for model in models:
    y_pred = model.predict(x_val)
    predictions.append(y_pred)

# Step 3: Take the majority class prediction (use the 'mode')
ensemble_predictions = mode(predictions, axis=0)[0]

# Step 4: Evaluate the prediction accuracy
ensemble_accuracy = accuracy_score(y_val, ensemble_predictions)

# Step 5: Benchmark/compare to a single model and see if the accuracy is better
# Evaluate the accuracy of a single model (e.g., the first model in the list)
single_model_accuracy = accuracy_score(y_val, predictions[0])

print("Ensemble Accuracy:", ensemble_accuracy)
print("Single Model Accuracy:", single_model_accuracy)


# Ploting the accuracy and loss over training iterations

In [ ]:
# list all data in history
print(model1.history.keys())

# summarize history for accuracy
plt.plot(model1.history['accuracy'])
plt.plot(model1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(model1.history['loss'])
plt.plot(model1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Print out the accuracy on the independent Test data set

In [ ]:
# print out the accuracy on independent test data
# score = model.evaluate(test_img, test_labels, verbose=0)
score1 = model.evaluate(x_val, y_val, verbose=0)
print("Test accuracy:", score1[1])
plt.imshow(x_val[0])

#Model 2

score2 = model2.evaluate(x_val, y_val, verbose=0)
print("Test accuracy:", score2[1])
plt.imshow(x_val[0])

#Model 3

score3 = model3.evaluate(x_val, y_val, verbose=0)
print("Test accuracy:", score3[1])
plt.imshow(x_val[0])

# Confusion Matrix

In [ ]:
# obtain model predictions and convert softmax outputs 0-1 to integer class label predictions
# Yhat = model.predict(test_img)
Yhat = model.predict(x_val)                    # predict model outputs on validation data as softmax outputs of probability of each class
Yhat_integer = np.argmax(Yhat, axis=1)            # obtain the most likely class prediction as the argument of the max softmax output
# Y_test_integer = np.argmax(test_labels, axis=1)
Y_test_integer = np.argmax(y_val, axis=1)   # obtain the true class as an integer

# calculate and plot confusion matrix
cm = confusion_matrix(Y_test_integer, Yhat_integer , normalize="pred")    # calculate the confusion matrix
plt.figure(2).set_figwidth(15)                                            # setup new figure
sns.heatmap(cm/np.sum(cm), annot=True, fmt=".2%", cmap="Blues",)          # plot the confusion matrix using the sns package
plt.title("Confusion Matrix", fontsize = 12)                              # title
plt.xlabel("Predicted Class", fontsize = 12)                              # xlabel
plt.ylabel("True Class", fontsize = 12)                                   # ylabel
plt.show()                                                                # show plot